In [234]:

from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import cv2
import numpy as np
import time

In [235]:
robot = Robot()

def motor_steer(speed, steering):
    if steering == 0:
        robot.left_motor.value = speed
        robot.right_motor.value = speed
        return
    elif steering > 0:
        steering = 100 - steering
        robot.right_motor.value = speed
        robot.left_motor.value = speed * steering / 100
        return
    elif steering < 0:
        steering = steering * -1
        steering = 100 - steering
        robot.left_motor.value = speed
        robot.right_motor.value = speed * steering / 100
        return

In [ ]:
camera = Camera.instance()
image = widgets.Image(format='jpeg', width=640, height=480)
steering = 0

def process_frame(change):
    Greendetected = False
    Blackdetected = False
    centerx_green = None
    centerx_blk = None
    global turn_flag
    
    frame = change['new']
    roi = frame[50:250, 0:640]
    roi_green = frame[0:70, 0:640]

    blurred = cv2.GaussianBlur(roi, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    mean_v = np.mean(v)
    _, Blackline = cv2.threshold(v, mean_v * 0.6, 255, cv2.THRESH_BINARY_INV)
    Blackline = cv2.bitwise_and(Blackline, cv2.inRange(s, 0, 80))

    hsv_green = cv2.cvtColor(roi_green, cv2.COLOR_BGR2HSV)
    h2, s2, v2 = cv2.split(hsv_green)
    mask_greenish = (h2 > 30) & (h2 < 90) & (s2 > 50)
    if np.any(mask_greenish):
        mean_h = np.mean(h2[mask_greenish])
        lower_h = max(mean_h - 10, 30)
        upper_h = min(mean_h + 10, 90)
    else:
        lower_h, upper_h = 30, 90
    Greensign = cv2.inRange(hsv_green, (lower_h, 60, 40), (upper_h, 255, 255))

    kernel = np.ones((6, 6), np.uint8)
    Blackline = cv2.erode(Blackline, kernel, iterations=5)
    Blackline = cv2.dilate(Blackline, kernel, iterations=6)
    Greensign = cv2.erode(Greensign, kernel, iterations=5)
    Greensign = cv2.dilate(Greensign, kernel, iterations=6)

    contours_blk, _ = cv2.findContours(Blackline.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_green, _ = cv2.findContours(Greensign.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours_green) > 0:
        Greendetected = True
        x_green, y_green, w_green, h_green = cv2.boundingRect(contours_green[0])
        centerx_green = x_green + (w_green // 2)
        cv2.line(roi_green, (centerx_green, 0), (centerx_green, 70), (0, 0, 255), 1)

    if len(contours_blk) > 0:
        Blackdetected = True
        largest = max(contours_blk, key=cv2.contourArea)
        x_blk, y_blk, w_blk, h_blk = cv2.boundingRect(largest)
        centerx_blk = x_blk + (w_blk // 2)
        cv2.line(roi, (centerx_blk, 50), (centerx_blk, 250), (255, 0, 0), 1)
        blackbox = cv2.minAreaRect(largest)
        (x_min, y_min), (w_min, h_min), ang = blackbox
        ang = int(ang)
        if ang < -45:
            ang = 90 + ang
        if w_min < h_min and ang > 0:
            ang = (90 - ang) * -1
        if w_min > h_min and ang < 0:
            ang = 90 + ang
        box = cv2.boxPoints(blackbox)
        box = np.int0(box)
        centertext1 = "angle = " + str(ang)
        cv2.putText(frame, centertext1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    if Greendetected and Blackdetected:
        if centerx_green > centerx_blk:
            cv2.putText(frame, "Turn Right", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        else:
            cv2.putText(frame, "Turn Left", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    if Greendetected and Blackdetected and not turn_flag:
        turn_flag = True
        if centerx_green > centerx_blk:
            time.sleep(0.15)
            motor_steer(0.2, 90)
            time.sleep(1.2)
            robot.stop()
        else:
            time.sleep(0.15)
            motor_steer(0.2, -90)
            time.sleep(1.2)
            robot.stop()
    elif not Greendetected:
        turn_flag = False

    if Blackdetected:
        setpoint = 120
        deviation = centerx_blk - setpoint
        centertext = "Deviation = " + str(deviation)
        cv2.putText(frame, centertext, (70, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        if abs(deviation) < 10 and abs(ang) < 7:
            steering = 0
        else:
            steering = deviation
        motor_steer(0.2, steering)
    else:
        robot.stop()

    image.value = bgr8_to_jpeg(frame)

display(image)
camera.observe(process_frame, names='value')

In [237]:
camera.stop()
robot.stop()